##### several pretty useful tutorial:
1:  https://3d-materials-lab.gitbook.io/3dmaterialslab-tutorials/defects/interpreting-defect-and-energy-level-diagrams

2: https://docs.google.com/presentation/d/1F6CUt2CIejgpLGcDgcVMgc5Mx6F569Gt/edit?pli=1&slide=id.p12#slide=id.p12

#### Prework1 download pydefect


In [ ]:
conda create –n my_pydefect  python=3.11 ##must >=3.7
conda  activate my_pydefect
pip install pydefect
(pip install vise)


In [ ]:
cd /home/kaijiz/job/1--research/0--vasp_calculation/01--Sb2Se3/5-defect_calcaulation
mkdir 01_Sb2Se3


#### Prework2 copy POTCAR files

we use vise (= vasp integrated supporting environment) code, which generates the input files for various tasks and exchange-correlation (XC) functionals. Vise relies on the pymatgen, 

In [ ]:
POTCAR path is : ‘ /sw/restricted/vasp/sw/potentials/potpaw_PBE.54’
cd ~
cd .config
vi .pmgrc.yaml 


#### This is what we should write in .pmgrc.yaml
PMG_DEFAULT_FUNCTIONAL: PBE_54    
(the version of potcar, I tried to change the name, found that it seems to have no effect, but to be safe, do not change it)

PMG_MAPI_KEY: DvETaAANdhxbBIXQa2x7uHWDQ2YwKMR5  

PMG_VASP_PSP_DIR: /home/kaijiz/job/1--research/0--vasp_calculation/01--Sb2Se3/5-defect_calcaulation/01_Sb2Se3 
(this is the position you want to copy all POTCARs to)


In [ ]:
cd /home/kaijiz/job/1--research/0--vasp_calculation/01--Sb2Se3/5-defect_calcaulation/01_Sb2Se3

mkdir POT_GGA_PAW_PBE_54 
(must be this name, vise will automatically find this directory and extract POTCAR from it)

cp –r /sw/restricted/vasp/sw/potentials/potpaw_PBE.54/* POT_GGA_PAW_PBE_54 


#### Prework3: Modify and Correct the mprester.py and mp_tools.py files
mprester.py is in:
/scratch/user/kaijiz/.conda/envs/pydefect/lib/python3.11/site-packages/mp_api/client/

mp_tools.py is in:
/scratch/user/kaijiz/.conda/envs/pydefect/lib/python3.11/site-packages/pydefect/util/

you can find new mprester.py in:
https://github.com/materialsproject/api/blob/main/mp_api/client/mprester.py

Find the DEFAULT_API_KEY line and hardcode the materials project key to the default API key (like the following):
DEFAULT_API_KEY = environ.get("MP_API_KEY", "DvETaAANdhxbBIXQa2x7uHWDQ2YwKMR5")




#### Step 1: Create the required directory tree

In [ ]:
cd /home/kaijiz/job/1--research/0--vasp_calculation/01--Sb2Se3/5-defect_calcaulation/01_Sb2Se3
mkdir unitcell
mkdir unitcell/structure_opt
mkdir unitcell/band
mkdir unitcell/dielectric
mkdir unitcell/dos
mkdir unitcell/relax
mkdir cpd
mkdir defect
mkdir defect/perfect

#### Step 2: download the initial structure

In [ ]:
cp 01_without_defect/01_Sb2Se3_scf/POSCAR unitcell/structure_opt/

#### relaxation
##### vise mainfunction 1: vs = vasp_set

In [ ]:
cd unitcell/structure_opt/
## use vise to automatically create all input files
vise vs -uis ENCUT 275.0
sbatch run_job

#### Step 3: Band, DOS, and Dielectric calculation of the relaxed Unit cell

In [ ]:
cp CONTCAR ../band/POSCAR
cp run_job ../band
cp CONTCAR ../dos/POSCAR
cp run_job ../dos
cp CONTCAR ../dielectric/POSCAR
cp run_job ../dielectric

In [ ]:
cd ../band
vise vs -uis ENCUT 520.0 -t band
(t:task)
sbatch run_job

##### plot band structure

In [ ]:
vise plot_band
grep E-fermi OUTCAR | tail -1

##### DOS calculation:

In [ ]:
cd ../dos
vise vs -uis ENCUT 275.0 LVTOT True LAECHG True -t dos 

sbatch run_job

##### plot dos

In [ ]:
vise plot_dos

#### Dielectric calculation:

In [ ]:
cd ../dielectric
vise vs -uis ENCUT 275.0 -t dielectric_dfpt
### NSW =0 !!!! for electronic dielectric calculation, or set IBRION =-1 
sbatch run_job

#### Gather unit cell calculation results for future reference
See the unitcell.yaml file to get the bandgap value (cbm - vbm = bandgap)

In [ ]:
cd ..
pydefect_vasp u -vb band/vasprun.xml -ob band/OUTCAR -odc dielectric/OUTCAR -odi dielectric/OUTCAR -n Sb2Se3
pydefect_print unitcell.yaml

#### Step 4: Create competing phases and run VASP calculation of the competing phases

the following code will extract corresponding structures from Materials Project

In [ ]:
cd ../cpd/
pydefect_vasp mp -e Sb Se --e_above_hull 0.0005
pydefect_vasp mp -e Sb --e_above_hull 0.0005
pydefect_vasp mp -e Se --e_above_hull 0.0005

##### if also considering impurities: Mg, all the atoms used in the structure should be added here.

In [ ]:
pydefect_vasp mp -e Mg --e_above_hull 0.0005
pydefect_vasp mp -e Sb Se Mg --e_above_hull 0.0005


#### Create VASP files for these competing phases and run the calculation

##### In each directory, there are POSCAR and prior_info.yaml. The prior_info.yaml contains some information retrieved from the Materials Project, which is useful for determining the conditions for the first-principles calculations via vise.

In [ ]:
#for i in *_*/;do cd $i; vise vs -uis ENCUT 520.0; cd ../;done
for i in *_*/;do cd $i; vise vs -uis ENCUT 416 NSW 200 EDIFFG -0.01 IVDW 11 LREAL F LVTOT T LVHAR T ISYM 0; cd ../;done

#Copy run.slurm to each competing phase folder and run VASP calculation

for dir in */;do cd $dir; cp ../../unitcell/band/run_job .; sbatch run_job; cd ../;done 

##### Generate the composition_energies.yaml file - which collects the total energies per calculated formula. Note, pydefect_print command is used to visualize the .json or .yaml files

In [ ]:
pydefect_vasp mce -d */
pydefect_print composition_energies.yaml

##### Create relative_energies.yaml and standard_energies.yaml. Read the files and store the energies for better understanding and future reference. The first command also generates convex_hull.pdf convex hall diagram.

standard_and_relative_energies (= sre) sub-command.

The standard_energies.yaml file collects the absolute energies under the standard states as shown.

while relative_energies.yaml includes the relative energies with respect to the standard states.

In [ ]:
pydefect sre
pydefect cv -t Sb2Se3
pydefect_print chem_pot_diag.json


#### To plot the diagram, use the plot_cpd (= pc) sub-command:

##### In the cpd.pdf plot, there will be points A, B, etc. meaning which element is Rich.
##### Remember this A,B, etc. points. Because these points will be used to plot chemical potential diagram (CPD)

In [ ]:
pydefect pc

#### Step 5: Creating supercell files and incorporate the required defect types and states

In [ ]:
cd ../defect
pydefect s -p ../unitcell/structure_opt/CONTCAR
pydefect_print supercell_info.json

# If one wants to know the conventional cell run the following (optional)
# vise si -p ../unitcell/structure_opt/CONTCAR -c

#### Incorporate defects (antisites and vacancies)

In [ ]:
pydefect ds

##### To see the defects created by the above command use thisL:


In [ ]:
pydefect_print defect_in.yaml

##### If we want to add dopants, we can type as follows:

In [ ]:
pydefect ds -d Ca

##### if want to change the oxidation of element, can type as follows:

In [ ]:
pydefect ds --oxi_states Mg 4

##### if we want to consider the interstitial defects, we needs to find the local minima of charge density. We can find relative information from DOS calculation: Get AECCAR0, AECCAR1, AECCAR2, and LOCPOT files generated by DOS calculation

In [ ]:
Ba_Sb1: [-2, -1, 0]
Ba_Sb2: [-2, -1, 0]
Ba_Se1: [0, 1, 2, 3, 4]
Ba_Se2: [0, 1, 2, 3, 4]
Ba_Se3: [0, 1, 2, 3, 4]
Sb_Se1: [-1, 0, 1, 2, 3]
Sb_Se2: [-1, 0, 1, 2, 3]
Sb_Se3: [-1, 0, 1, 2, 3]
Se_Sb1: [-3, -2, -1, 0, 1]
Se_Sb2: [-3, -2, -1, 0, 1]
Va_Sb1: [-3, -2, -1, 0, 1]
Va_Sb2: [-3, -2, -1, 0, 1]
Va_Se1: [-1, 0, 1, 2]
Va_Se2: [-1, 0, 1, 2]
Va_Se3: [-1, 0, 1, 2]

In [ ]:
cp ../unitcell/dos/AECCAR0 .
cp ../unitcell/dos/AECCAR1 .
cp ../unitcell/dos/AECCAR2 .
cp ../unitcell/dos/LOCPOT .
pydefect_vasp le -v AECCAR{0,2} -i all_electron_charge
## this will create volumetric_data_local_extrema.json, 
## which shows the local minima of the charge density as follows.

#### Attention !!!!  : if you are not satisfied with the interstitial you just created and want to choose another position or choose different amount. Please delete 'supercell_info.json' and use 'pydefect s -p ../unitcell/structure_opt/CONTCAR' to create a new 'supercell_info.json' and then add new interstitials you want by using the following commands. Otherwise, you will got many duplicate interstitials.

In [ ]:
pydefect_util ai --local_extrema volumetric_data_local_extrema.json -i 1 2

##### create defect_in.yaml file which has informations of all defects
###### if you want to change the charge state of defect, just 'vi defect_in.yaml'

In [ ]:
pydefect ds

##### create all defect structures based on defect_in.yaml


In [ ]:
pydefect_vasp de

#### Attention!! there is a bug in 'pydefect_vasp de', when the charge state=0, the prior_info.yaml in each folder will not show information correctly. To correct this, we cam use following command:

In [ ]:
for i in *_0/; do echo "charge: 0" > "$i/prior_info.yaml"; done

#### Create corresponding input files and run_job and then submit the job

In [ ]:
for i in */;do cd $i; vise vs -t defect -uis ENCUT 275 NSW 200 EDIFFG -0.01; cd ../;done

#for i in */;do cd $i; vise vs -t defect -uis ENCUT 400 NSW 200 EDIFFG -0.01 IVDW 11 LREAL F LVTOT T LVHAR T ISYM 0; cd ../;done
for i in */;do cd $i; vise vs -t defect -uis ENCUT 416 PREC Accurate  NSW 200 EDIFFG -0.01 EDIFF 1e-7 LASPH True IVDW 11 LREAL False ISYM 0; cd ../;done
for i in */;do cd $i; cp ../run_job .; cd ../;done
for dir in */;do cd $dir; sbatch run_job; cd ..; done

##### Parsing all calculation results and generate the calc_results.json that contains the first-principles calculation results related to the defect properties.


In [ ]:
pydefect_vasp cr -d *_*/ perfect


#### Step 6: Add energy corrections
##### The total energies for charged defects are not properly estimated due to interactions between a defect, its images, and background charge. 
##### The following command creates correction.json and correction.pdf files in each of the defect directory

In [ ]:
pydefect efnv -d *_*/ -pcr perfect/calc_results.json -u ../unitcell/unitcell.yaml

##### The following command creates defect_structure_info.json files to analyze the defect local structures [in each defect directory]

In [ ]:
pydefect dsi -d *_*/

##### The following command creates VESTA file (defect.vesta) for analyzing the defect structure files [in each defect directory]

In [ ]:
pydefect_util dvf -d *_*/ 

#### Step 7: Plot defect formation energy diagram

##### Creates defect_energy_info.yaml in each of the defect directories

In [ ]:
pydefect dei -d *_*/ -pcr perfect/calc_results.json -u ../unitcell/unitcell.yaml -s ../cpd/standard_energies.yaml

##### Create a defect_energy_summary.json file with the defect_energy_summary (= des) sub-command. [in the parent defect folder]

In [ ]:
pydefect des -d *_*/ -u ../unitcell/unitcell.yaml -pbes perfect/perfect_band_edge_state.json -t ../cpd/target_vertices.yaml

#### Create the calc_summary.json file with the calc_summary (= cs) sub-command [in the parent defect directory] (still test version)

In [ ]:
pydefect cs -d *_*/ -pcr perfect/calc_results.json

#### get the plot

In [ ]:
pydefect pe -d defect_energy_summary.json -l A --no_label_line
pydefect pe -d defect_energy_summary.json -l B --no_label_line